<a href="https://colab.research.google.com/github/NataliaNadolna/LLM/blob/main/LLM_as_a_Judge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Wczytanie modelu sędziego

In [1]:
!pip install -q -U "google-generativeai>=0.7.2"

In [2]:
import google.generativeai as genai

In [11]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [12]:
version = 'models/gemini-1.5-flash' # @param ["models/gemini-1.5-flash", "models/gemini-1.5-pro", "models/gemini-1.0-pro"]
model = genai.GenerativeModel(version)

In [14]:
response = model.generate_content("Cześć, co słychać?")
print(response.text)

Cześć! U mnie wszystko w porządku, a u Ciebie?



Wczytanie danych

In [22]:
import pandas as pd

data = {'Prompt': ['Kto ma imieniny 30 lutego?',
                   'W którym roku XIV wieku doszło do słynnej bitwy między wojskiem Watykanu i Chin na terenie obecnej Polski?',
                   'Agnieszka twierdzi, że urodziła się 29 lutego 1981 roku. Zakładając, że to prawda, ile razy obchodziła swoje urodziny w dniu 29 lutego?'],
        'Model_answer': ['W Polsce nikt nie obchodzi imienin 30 lutego, ponieważ w lutym nie ma "pełnego" roku, czyli dwunastu miesięcy. Data 30 lutego występuje w kalendarzach opracowanych na podstawie astronomicznych przesileń i często jest traktowana jako dzień przestępny w kalendarzach o charakterze świątecznym lub religijnym. W kalendarzu gregoriańskim, który jest używany w większości krajów, 30 lutego jest uważany za dzień przestępny i jest mało prawdopodobne, aby większość ludzi miała imieniny w tym dniu.',
                         'W 1331 roku doszło do słynnej bitwy między wojskiem Watykanu a Chin na terenie obecnej Polski. W rzeczywistości jednak nie było żadnej bitwy ani żadnych wojsk Watykanu czy Chin w Polsce w XIV wieku. Jest to prawdopodobnie błąd w datacji lub w opisie zdarzenia.',
                         'Agnieszka urodziła się 29 lutego 1981 roku. W Polsce, co 4 lata, ten dzień nie jest obchodzony. Występuje on tylko w latach, które dzielą się przez 4, np. w 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020, i 2024. W tych latach, Agnieszka obchodzi swoje urodziny dwa razy, w dniu 29 lutego i w dniu 31 stycznia (w "przestępnym" roku). Jeśli urodziny wypadają w "przestępnym" roku, formalnie obchodzi się je dwa dni wcześniej w stosunku do "nieprzestępnego" roku, czyli 28'],
        'True_answer': ['Nikt, bo nie ma takiego dnia.',
                        'Nie było takiej bitwy.',
                        'Ani razu, bo nie mogła się wtedy urodzić.']}
df = pd.DataFrame(data)

print(df)

                                              Prompt  \
0                         Kto ma imieniny 30 lutego?   
1  W którym roku XIV wieku doszło do słynnej bitw...   
2  Agnieszka twierdzi, że urodziła się 29 lutego ...   

                                        Model_answer  \
0  W Polsce nikt nie obchodzi imienin 30 lutego, ...   
1  W 1331 roku doszło do słynnej bitwy między woj...   
2  Agnieszka urodziła się 29 lutego 1981 roku. W ...   

                                 True_answer  
0              Nikt, bo nie ma takiego dnia.  
1                     Nie było takiej bitwy.  
2  Ani razu, bo nie mogła się wtedy urodzić.  


In [23]:
print(df.loc[0, 'Prompt'])

Kto ma imieniny 30 lutego?


In [24]:
print(df['Prompt'])

0                           Kto ma imieniny 30 lutego?
1    W którym roku XIV wieku doszło do słynnej bitw...
2    Agnieszka twierdzi, że urodziła się 29 lutego ...
Name: Prompt, dtype: object


Generowanie odpowiedzi

In [28]:
prompt = f"ROLA: Jesteś rzetelnym asystentem oceniającym. ZAWSZE ściśle trzymasz się instrukcji. Opierasz się wyłącznie na materiałach zaprezentowanych przez użytkownika. Wykonujesz TYLKO swoje ZADANIE, pomijasz dodatkowe komentarze. ZADANIE: Twoje zadanie polega wyłącznie na dostarczeniu obiektywnej OCENY odpowiedzi zawartej w treści #OCENIANA_ODPOWIEDZ#. Porównaj ją z odpowiedzią zawartą w #PRAWIDLOWA_ODPOWIEDZ#. Są to odpowiedzi na pytanie zawarte w punkcie #PYTANIE#. Wyślij wyłącznie OCENĘ w skali od 0 do 5, gdzie 0 to oceniana odpowiedź zupełnie niezgodna z prawidłową odpowiedzią, a 5 to oceniana odpowiedź całkowicie zgodna z prawidłową odpowiedzią. To bardzo ważne - nie wysyłaj nic poza cyfrą od 0 do 5, pomiń dodatkowe znaki i komentarze. Uwaga: Oceniana odpowiedź może zawierać rozszerzenie tematu lub dodatkowe informacje - nie powinieneś obniżać za to OCENY. Najważniejsze, żeby oceniana gdzieś w swojej treści zawierała informacje z prawidłowej odpowiedzi. PYTANIE: {df.loc[0, 'Prompt']} OCENIANA_ODPOWIEDZ: {df.loc[0, 'Model_answer']} PRAWIDLOWA_ODPOWIEDZ: {df.loc[0, 'True_answer']}"

In [26]:
response = model.generate_content(prompt)
print(response.text)

4



In [30]:
for index, row in df.iterrows():
  prompt = f"ROLA: Jesteś rzetelnym asystentem oceniającym. ZAWSZE ściśle trzymasz się instrukcji. Opierasz się wyłącznie na materiałach zaprezentowanych przez użytkownika. Wykonujesz TYLKO swoje ZADANIE, pomijasz dodatkowe komentarze. ZADANIE: Twoje zadanie polega wyłącznie na dostarczeniu obiektywnej OCENY odpowiedzi zawartej w treści OCENIANA_ODPOWIEDZ. Porównaj ją z odpowiedzią zawartą w PRAWIDLOWA_ODPOWIEDZ. Są to odpowiedzi na pytanie zawarte w punkcie PYTANIE. Wyślij wyłącznie OCENĘ w skali od 0 do 5, gdzie 0 to oceniana odpowiedź zupełnie niezgodna z prawidłową odpowiedzią, a 5 to oceniana odpowiedź całkowicie zgodna z prawidłową odpowiedzią. To bardzo ważne - nie wysyłaj nic poza cyfrą od 0 do 5, pomiń dodatkowe znaki i komentarze. Uwaga: Oceniana odpowiedź może zawierać rozszerzenie tematu lub dodatkowe informacje - nie powinieneś obniżać za to OCENY. Najważniejsze, żeby oceniana gdzieś w swojej treści zawierała informacje z prawidłowej odpowiedzi. \nPYTANIE: {row['Prompt']} \nOCENIANA_ODPOWIEDZ: {row['Model_answer']} \nPRAWIDLOWA_ODPOWIEDZ: {row['True_answer']}"
  print(prompt)
  response = model.generate_content(prompt)
  print(response.text)

ROLA: Jesteś rzetelnym asystentem oceniającym. ZAWSZE ściśle trzymasz się instrukcji. Opierasz się wyłącznie na materiałach zaprezentowanych przez użytkownika. Wykonujesz TYLKO swoje ZADANIE, pomijasz dodatkowe komentarze. ZADANIE: Twoje zadanie polega wyłącznie na dostarczeniu obiektywnej OCENY odpowiedzi zawartej w treści OCENIANA_ODPOWIEDZ. Porównaj ją z odpowiedzią zawartą w PRAWIDLOWA_ODPOWIEDZ. Są to odpowiedzi na pytanie zawarte w punkcie PYTANIE. Wyślij wyłącznie OCENĘ w skali od 0 do 5, gdzie 0 to oceniana odpowiedź zupełnie niezgodna z prawidłową odpowiedzią, a 5 to oceniana odpowiedź całkowicie zgodna z prawidłową odpowiedzią. To bardzo ważne - nie wysyłaj nic poza cyfrą od 0 do 5, pomiń dodatkowe znaki i komentarze. Uwaga: Oceniana odpowiedź może zawierać rozszerzenie tematu lub dodatkowe informacje - nie powinieneś obniżać za to OCENY. Najważniejsze, żeby oceniana gdzieś w swojej treści zawierała informacje z prawidłowej odpowiedzi. 
PYTANIE: Kto ma imieniny 30 lutego? 
OC